In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
from pprint import pprint
from openai.types.chat import ChatCompletionToolParam
import json

df = pd.read_excel('商品レビュー.xlsx', sheet_name='商品レビュー')

tools=[
  ChatCompletionToolParam({
    "type": "function",
    "function": {
      "name": "get_reviews",
      "description": "商品名「product」と、商品ごとの褒められた点「goods」と指摘された問題点「bads」を結合した結果を返す",
      "parameters": {
        "type": "object",
        "properties": {
          "product_data": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "product": {
                  "type": "string",
                  "description": "商品名",
                },
                "goods": {
                  "type": "string",
                  "description": "特定商品の褒められた点をカンマ区切りで並べた文字列。",
                },
                "bads": {
                  "type": "string",
                  "description": "特定商品の指摘された問題点をカンマ区切りで並べた文字列。",
                },
              }
            },
          },
        },
        "required": ["product", "goods", "bads"],
      },
    },
  })
]

def llm_analyz(prompt_text):
    role = "あなたはマーケティング分野に精通したデータサイエンティストです。商品レビューを分析し、商品ごと褒められた点と指摘された問題点を必ず抽出してください"
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": prompt_text},
        ],
        tools=tools,
        tool_choice={
            "type":"function",
            "function":{"name":"get_reviews"}
        },
    )
    return response

df_mean = df.groupby(['商品名', 'カテゴリ'])['評価（1-5）'].mean().round(2).reset_index()
df_mean.columns = ['商品名', 'カテゴリ', '平均評価']

sales_data_text = df[['商品名', 'レビュー本文']].astype(str).to_string(index=False)
prompt_text = f"商品レビュー:\n{sales_data_text}\n商品レビューの傾向を分析してください。"
response = llm_analyz(prompt_text)
tool = response.choices[0].message.tool_calls[0]
data = json.loads(tool.function.arguments)
df_reviews = pd.DataFrame(data['product_data'])

df_result = pd.merge(df_mean, df_reviews, left_on='商品名', right_on='product').drop(columns='product')
df_result 



,商品名,カテゴリ,平均評価,goods,bads
0,オーガニックコーヒー豆,食品,3.67,"香りが豊か,朝の一杯が楽しみ,酸味と苦味のバランスが絶妙,品質が高い,美味しい","値段が高め,届いた豆が酸化していた,鮮度管理に問題がある"
1,スマートウォッチZ2,電子機器,3.00,"デザインがスタイリッシュ,健康管理機能が充実,睡眠トラッキングが便利","バッテリーの持ちが悪い,動作が遅い,防水機能が効果的でない,カスタマーサポートが悪い"
2,プロテインシェーカー,スポーツ,3.00,"漏れ防止設計が完璧,洗いやすく衛生的","蓋が割れやすい,プラスチックの質が悪い"
3,ヨガマットPro,スポーツ,4.00,"クッション性が高い,膝や手首への負担が少ない,滑り止めがしっかりしている,厚みがあって快適","重くて持ち運び不便,外出用には向かない,最初のゴムの匂いが強い"
4,ワイヤレスイヤホンX1,電子機器,3.67,"音質が素晴らしい,低音がしっかり出ている,装着感が快適,バッテリーが一日中持つ,コスパが良い...","接続が不安定,音質には問題ないがBluetoothの品質に問題,ケースが壊れやすい"
5,保湿クリームDX,美容,3.67,"肌がしっとり,翌朝のモチモチ感が続く,敏感肌でも刺激がない,保湿力が高い","べたつきが気になる,香料が強すぎて頭痛がする"
